In [2]:
import pandas as pd

In [26]:
contra1 = pd.read_csv('unlabeled/XMLProduct_contrainidcations.csv')
print("contraindication first:",len(contra1))
contra = contra1.drop_duplicates(subset=['Label_ID', 'DO_ID'], keep = 'first').reset_index(drop = True)
print("contraindication second:",len(contra))
contra.sample()

contraindication first: 59748
contraindication second: 41795


,Label_ID,From,To,Type,Annotation,Context,DO_ID,DB_ID,DrugName,Drug_Brand_Name,Set_ID,Section
13386,e870a709-d13f-4ec3-86b1-cf3d8d79e113.xml,308,315,PREF,SYNDROME,Hypersensitivity to venlafaxine hydrochloride ...,DOID_225,DB00285,VENLAFAXINE,Venlafaxine Hydrochloride,e5a31f69-c237-49b7-83a2-d04eebbc30ce,Contraindication


In [17]:
treat1 = pd.read_csv('unlabeled/XMLProduct_indications.csv')
print("indication first:",len(treat1))
treat = treat1.drop_duplicates(subset=['Context' ,'DO_ID', 'DB_ID'], keep = 'first').reset_index(drop = True)
print("indication second:",len(treat))

indication first: 127326
indication second: 36198


In [45]:
#treat[treat.duplicated(subset=['Context', 'DO_ID'],keep=False)].sort_values(by=['Annotation'])

In [44]:
treat.Type.value_counts()

PREF    30224
SYN      5974
Name: Type, dtype: int64

In [3]:
contra = pd.read_csv('unlabeled/XMLProduct_contrainidcations.csv')
treat = pd.read_csv('unlabeled/XMLProduct_indications.csv')
print("contraindication first:",len(contra))
print("indication first:",len(treat))
print(contra['DO_ID'].isna().sum())
print(treat['DO_ID'].isna().sum())
contra = contra.drop_duplicates(subset=['Context', 'DO_ID'], keep = 'first').reset_index()
treat = treat.drop_duplicates(subset=['Context', 'DO_ID'], keep = 'first').reset_index()
print("contraindication after",len(contra))
print("indication after",len(treat))
print(contra['DO_ID'].isna().sum())
print(treat['DO_ID'].isna().sum())

contraindication first: 59748
indication first: 127326
0
0
contraindication after 17850
indication after 36192
0
0


In [4]:
frames = [contra, treat]

merged= pd.concat(frames).reset_index(drop = True)

In [5]:
merged.shape

(54042, 13)

In [6]:
#merged = merged[['Label_ID', 'Annotation', 'Context', 'DO_ID','DB_ID', 'DrugName', 'Drug_Brand_Name', 'Section']]
#merged.sample()

In [7]:
df = merged.drop_duplicates(subset=['Context', 'DO_ID'], keep='first').reset_index()
df.shape

(54040, 14)

In [12]:
# Clean the text

def lower (x):
    return x.lower()

# remove punctuation
def remove_symbols(x):
    for char in x:
            if char not in ['1','2','3','4','5','6','7','8','9','q','Q','w','W', 'e','E','r','R','t','T','y','Y','U','u','i','I','O','o','P','p','A','a','S','s','D','d','F','f','G','g','H','h','J','j','K','k','L','l','Z','z','X','x','C','c','V','v','B','b','N','n','M','m']:
                x = x.replace(char, ' ')
    return x

def check(x, text):
    return x in text

def remove_xml_symbols(text):
    text1 = text.replace('\n', '  ')
    clean_text = text1.replace('\r', '  ')
    return clean_text

def remove_spaces(text):
    clean_text = ' '.join(text.split())
    return clean_text


In [58]:
def disease_mention(x):
    f= int(x['From'])
    t= int(x['To'])
    text = x['Context']
    return text[f-1:t]

In [59]:
df.sample()

,level_0,index,Label_ID,From,To,Type,Annotation,Context,DO_ID,DB_ID,DrugName,Drug_Brand_Name,Set_ID,Section
37729,37729,73246,75b26af4-19fb-cf08-e053-2991aa0a556d.xml,952,958,PREF,DISEASE,Therapy with lipid-altering agents should be o...,DOID_4,DB00627,NIACIN,NIACIN,09bccfc6-71eb-420f-92d3-cd21789a0dbc,Indication


In [61]:
df['disease'] = df.apply(disease_mention, axis='columns')

In [62]:
df['Context'] = df['Context'].apply(remove_xml_symbols)
df['Context'] = df['Context'].apply(remove_symbols)
df['Context'] = df['Context'].apply(remove_spaces)
df['Context'] = df['Context'].apply(lower)

df['disease'] = df['disease'].apply(remove_xml_symbols)
df['disease'] = df['disease'].apply(remove_symbols)
df['disease'] = df['disease'].apply(remove_spaces)
df['disease'] = df['disease'].apply(lower)


In [64]:
df.to_csv('unlabeled/raw_cleaned.csv')